<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#|-Preliminaries" data-toc-modified-id="|-Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>| Preliminaries</a></span></li><li><span><a href="#|-Cleaning" data-toc-modified-id="|-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>| Cleaning</a></span></li><li><span><a href="#|-Named-Entity-Recognition-w/-spaCy" data-toc-modified-id="|-Named-Entity-Recognition-w/-spaCy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>| Named Entity Recognition w/ spaCy</a></span><ul class="toc-item"><li><span><a href="#Split-first,-middle-and-last-name-in-different-cols" data-toc-modified-id="Split-first,-middle-and-last-name-in-different-cols-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Split first, middle and last name in different cols</a></span></li><li><span><a href="#|-Enrich-first-name-if-only-last-name-is-mentioned" data-toc-modified-id="|-Enrich-first-name-if-only-last-name-is-mentioned-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>| Enrich first name if only last name is mentioned</a></span></li></ul></li><li><span><a href="#|-Gender-Guesser" data-toc-modified-id="|-Gender-Guesser-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>| Gender Guesser</a></span></li><li><span><a href="#|-Group-per-article-and-get-share-of-gender-per-article" data-toc-modified-id="|-Group-per-article-and-get-share-of-gender-per-article-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>| Group per article and get share of gender per article</a></span></li><li><span><a href="#|-Wikidata-Query" data-toc-modified-id="|-Wikidata-Query-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>| Wikidata Query</a></span></li></ul></div>

# | Preliminaries

In [1]:
import os, json
import pandas as pd
import numpy as np
import regex as re

#from bs4 import BeautifulSoup as bs

# SpaCy 
import spacy
import en_core_web_trf

nlp = spacy.load("en_core_web_trf")

from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

import warnings
warnings.filterwarnings('ignore') # (action='once')


In [2]:
%%time

'''with open('.../Downloads/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

df = pd.DataFrame(data)'''


'''filename = ".../Downloads/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pd.read_json(filename)'''

df = pd.read_json(".../Downloads/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl", lines=True,nrows=10000)

CPU times: user 510 ms, sys: 124 ms, total: 634 ms
Wall time: 640 ms


# | Cleaning

In [3]:
with pd.option_context('display.max_colwidth', None):
  display(df[:2])

id  \
0  b2e89334-33f9-11e1-825f-dabc29fd7071   
1  749ec5b2-32f5-11e1-825f-dabc29fd7071   

                                                                                                                                            article_url  \
0  https://www.washingtonpost.com/sports/colleges/danny-coale-jarrett-boykin-are-a-perfect-1-2-punch-for-virginia-tech/2011/12/31/gIQAAaW4SP_story.html   
1                                         https://www.washingtonpost.com/local/bubbly-planet-venus-starts-off-new-year/2011/12/30/gIQAlbm4SP_story.html   

                                                                   title  \
0  Danny Coale, Jarrett Boykin are a perfect 1-2 punch for Virginia Tech   
1                                Bubbly planet Venus starts off New Year   

                      author  published_date  \
0             Mark Giannotto   1325376562000   
1  Blaine P. Friedlander Jr.   1325376729000   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [4]:
def cleaning_for_NER(df):
    
    '''dictionary = {
              "id"                     : "oid",
              "name"                   : "article_title",
              'teaser'                 : 'article_teaser',
              "text"                   : "article_text",
              "regions"                : "cms_regions"
                }
    '''
    df = df.copy()
    #df.rename(columns=dictionary, inplace=True)  # rename columns
    df.drop_duplicates(subset=['id'], inplace=True)

    # Remove "\n" in article_text
    df["content"] = df["content"].apply(lambda x: re.sub("\{'content': '"," ", str(x)))
 
    return df    

In [5]:
# Safe cleaned data in new df
df = cleaning_for_NER(df)

In [6]:
#df.drop(columns = ['contents'], inplace = True)

In [7]:
# drop empty rows
nan_value = float("NaN")
df.replace("", 'no title', inplace=True)
df["title"].replace(np.NaN, 'no title', inplace=True)

#df.dropna(subset = ["title"], inplace=True)

In [8]:
# Combine column "title" & "content" into "merged_total_text":
df['merged_total_text'] = df.apply(lambda r: r['title'] + r['content'], axis=1)

In [36]:
print(df.head(2))

                                     id  \
0  b2e89334-33f9-11e1-825f-dabc29fd7071   
1  b2e89334-33f9-11e1-825f-dabc29fd7071   

                                         article_url  \
0  https://www.washingtonpost.com/sports/colleges...   
1  https://www.washingtonpost.com/sports/colleges...   

                                               title          author  \
0  Danny Coale, Jarrett Boykin are a perfect 1-2 ...  Mark Giannotto   
1  Danny Coale, Jarrett Boykin are a perfect 1-2 ...  Mark Giannotto   

   published_date                                           contents     type  \
0   1325376562000  [{'content': 'Colleges', 'mime': 'text/plain',...  article   
1   1325376562000  [{'content': 'Colleges', 'mime': 'text/plain',...  article   

                source                                            content  \
0  The Washington Post  [ Colleges', 'mime': 'text/plain', 'type': 'ki...   
1  The Washington Post  [ Colleges', 'mime': 'text/plain', 'type': 'ki...   

        

# | Named Entity Recognition w/ spaCy

In [10]:
df_1 = df.copy()

In [11]:
# write entity in parsed_articles 
import tqdm
parsed_articles = []
for article, id in tqdm.tqdm(zip(df_1['content'], df_1['id']), total=len(df_1)):
    parsed_articles.extend([[id, str(entity), entity.label_] for entity  in nlp(article).ents])

100%|██████████████████████████████████| 10000/10000 [15:03:10<00:00,  5.42s/it]


In [12]:
# create dfs from entities
df_2 = pd.DataFrame(parsed_articles, columns=['id', 'content', 'entity_type'])

# rename columns for one df
df_2.rename(columns={'content': 'entity'}, inplace=True)

In [14]:
# merge back to initial df
df = (pd.merge(df_1, df_2, on='id'))

In [15]:
df.to_pickle(r"Washington_Post_NER_all_10K.pkl")

In [18]:
df_ner = df.copy()

In [158]:
df = pd.read_pickle("/Users/landsiedelj/Downloads/Washington_Post_NER_all_10K.pkl")

In [159]:
# NER 'PERSON' STARTS HERE
df = (df.loc[df['entity_type'].isin(['PERSON'])])

In [160]:
df["entity"] = df['entity'].str.replace('[\[\]\"\'\d\,\<\/]','')


In [162]:
# Replace artist names (Sting, Bono, Cher etc.) with name according to wikipedia
# TO do: write into dict?
df['entity'] = df['entity'].str.replace('JFK','John Fitzgerald Kennedy')
df['entity'] = df['entity'].str.replace('FDR','Franklin Delano Roosevelt')
df['entity'] = df['entity'].str.replace('Sting','Gordon Matthew Sumner')
df['entity'] = df['entity'].str.replace('Bono','Paul David Hewson')
df['entity'] = df['entity'].str.replace('Cher','Cherilyn Sarkisian')
df['entity'] = df['entity'].str.replace('Madonna','Madonna Louise Ciccone')
df['entity'] = df['entity'].str.replace('Adele','Adele Laurie Adkins')
df['entity'] = df['entity'].str.replace('Eminem','Marshall Bruce Mathers')
df['entity'] = df['entity'].str.replace('Beyonce','Beyoncé Knowles-Carter')
df['entity'] = df['entity'].str.replace('Blaine Friedlander','Blaine P. Friedlander Jr.')

In [174]:
df[:40]

,id,article_url,title,author,published_date,contents,type,source,content,merged_total_text,entity,entity_type,first_name,middle_name,last_name,gender_guesser
0,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Danny Coale,PERSON,Danny,,Coale,male
1,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Jarrett Boykin,PERSON,Jarrett,,Boykin,male
4,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Danny Coale,PERSON,Danny,,Coale,male
8,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Jarrett Boykin,PERSON,Jarrett,,Boykin,male
9,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",John Bazemore,PERSON,John,,Bazemore,male
13,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Danny Coale,PERSON,Danny,,Coale,male
17,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Jarrett Boykin,PERSON,Jarrett,,Boykin,male
18,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",John Bazemore,PERSON,John,,Bazemore,male
26,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Kevin Sherman,PERSON,Kevin,,Sherman,male
27,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 't

In [164]:
df = df[df['author'] != df['entity']] # if author is in col entity -> remove, we only want "Person" from within the article
# To do: Blaine P. Friedlander Jr. vs. Blaine Friedlander 

## Split first, middle and last name in different cols

In [165]:
# Write first, middle and last name in different cols
# to do: split middle and last name does not work
def split_name(df, var):
    sub_df = df[var].str.split('\\s+', expand=True)
    result = []

    for _, row in sub_df.iterrows():
        info = {'first_name': '', 'middle_name': '', 'last_name': ''}
        n = row.count()

        if n == 0:
            pass
        elif n == 1:
            info['last_name'] = row.iloc[0]
        elif n == 2:
            info['first_name'], info['last_name'] = row.iloc[:2]
        else:
            info['first_name'] = row.iloc[0]
            info['last_name'] = row.iloc[-1]
            info['middle_name'] = ' '.join([(string or '') for string in row.iloc[1:-1]])
        result.append(info)
    return pd.DataFrame(result, index=df.index)

df_names = split_name(df, 'entity')
df = df.join(df_names)

## | Enrich first name if only last name is mentioned

In [205]:
names = df.groupby('id')['entity'].apply(lambda x: list(np.unique(x)))
# To do: if only one token ("Obama") drop or don't write to list

In [188]:
# function for enriching first name if only last name is mentioned
from tqdm import tqdm
tqdm.pandas()
def enrich_firstname(row):
    
    entity = row['entity']#.copy()
    firstname = row['first_name']#.copy()
    lastname = row['last_name']#.copy()
    
    
    #if len(firstname) == 0:
        # look in list per id for key and write value?
    # elif 
    # bestehende Vornamen nicht überschreiben
    #else:
    #    row['first_name'] = 'unknown'
    return row

df_test = df_test.progress_apply(enrich_firstname, axis=1)


100%|██████████████████████████████████████| 1000/1000 [00:01<00:00, 981.24it/s]


# | Gender Guesser

In [169]:
# !pip install gender_guesser    
import gender_guesser.detector as gender
gd = gender.Detector()
df['gender_guesser'] = df['first_name'].apply(str.capitalize).map(lambda x: gd.get_gender(x))

> __unknown__ (name not found), __andy__ (androgynous), __male__, __female__, __mostly_male__, or __mostly_female__. The difference between andy and unknown is that the former is found to have the same probability to be male than to be female, while the later means that the name wasn’t found in the database.

In [170]:
df.gender_guesser.value_counts()

unknown          105606
male              46936
female            15921
mostly_male        2884
mostly_female      2206
andy               1139
Name: gender_guesser, dtype: int64

In [134]:
df.loc[df['gender_guesser'] == 'unknown'][:40]

,id,article_url,title,author,published_date,contents,type,source,content,merged_total_text,entity,entity_type,first_name,middle_name,last_name,gender_guesser
27,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Boykin,PERSON,,,Boykin,unknown
28,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Coale,PERSON,,,Coale,unknown
32,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Macho Harris,PERSON,Macho,,Harris,unknown
33,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Boykin,PERSON,,,Boykin,unknown
34,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Coale,PERSON,,,Coale,unknown
35,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Boykin,PERSON,,,Boykin,unknown
43,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Coale,PERSON,,,Coale,unknown
48,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Boykin,PERSON,,,Boykin,unknown
51,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Danny,PERSON,,,Danny,unknown
54,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Beamer,PER

In [172]:
# df.first_name.value_counts()[:40]

In [ ]:
# to do:
# Namen cleanen
# weitere lib für gender identification einbauen
# function schreiben: wenn nur Nachname genannt, schaue in Liste/anderer Zeile pro Artikel, ob Name schon genannt, dann Vorname auffüllen
# oder
# wenn Obama, Santorum, Gingrich, etc. dann aus erstellter Liste oder via Wikidata mit Vornamen auffüllen
# # dominique rodgers cromartie etc. (famous people)  -> Wikidata? 

# | Group per article and get share of gender per article

In [ ]:
# next

# | Wikidata Query

In [138]:
# oder das https://stackoverflow.com/questions/51419785/extract-data-from-wikidata-in-python

import requests

sparql_query = """
        prefix schema: <http://schema.org/>
        SELECT ?item ?occupation ?genderLabel ?bdayLabel
        WHERE {
            <https://en.wikipedia.org/wiki/Angela_Merkel> schema:about ?item .
            ?item wdt:P21 ?gender .
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        }
    """

In [139]:
url = 'https://query.wikidata.org/sparql'

# sleep(2)
r = requests.get(url, params={'format': 'json', 'query': sparql_query})

In [140]:
url = 'https://query.wikidata.org/sparql'

r = requests.get(url, params={'format': 'json', 'query': sparql_query})
data = r.json()

print(data['results']['bindings'])

[{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q567'}, 'genderLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'female'}}]
